In [97]:
import requests
import json
from dotenv import load_dotenv
import os
import dspy

In [102]:
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [70]:
response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": "perplexity/sonar-small-chat", # Optional
    "messages": [
      {"role": "user", "content": "Give me links to current news articles related to this tweet: January 6th was a dark day for America. We must never forget the insurrection." \
        + "Make sure to format the output so that there is a list of dictionaries, with each dictionaries containing a title and a link to the article. Only return with the object in string format and no other text at all outside of it. \
          Do not have the string 'json' in the output"}
    ]
  })
)
data = json.loads(response.text)
message_content = data['choices'][0]['message']['content']
# lines = message_content.split('\n\n')
# out = json.dumps(lines)
message_content


'You can ignore this.'

In [34]:
entries

[{'title': 'Biden Vows to Bail Out Struggling Former Coronavirus Hotspots',
  'link': 'https://news.bloomberglaw.com/coronavirus-covid-19/biden-vows-to-bail-out-struggling-former-coronavirus-hotspots'},
 {'title': 'Donald Trump, 55 Other Lawmakers Testify They Were Treated Badly in Capitol Strife',
  'link': 'https://www.nytimes.com/2021/06/30/us/politics/trump-testimony-capitol-riot.html'},
 {'title': "Trump's CPAC Speech Oozes of 2020, Offers Little About 2024",
  'link': 'https://www.politico.com/news/2021/07/11/trump-cpac-speech-495378'},
 {'title': 'US Military to Leave Afghanistan by September 11',
  'link': 'https://www.npr.org/2021/06/19/1008715273/us-military-to-leave-afghanistan-by-september-11'},
 {'title': 'Biden Finds Bipartisan Compromise on Borders, Major Investment Roads and Bridges',
  'link': 'https://www.politico.com/news/2021/07/06/biden-republican-compromise-494754'}]

In [26]:
#loop through indexes 2, 4, 6, 8, 10:
for i in range(2, len(lines), 2):
    title = lines[i]
    link = lines[i+1]
    print(f"{title}: {link}")

In [202]:
def text_to_politics(text):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        },
        data=json.dumps({
            "model": "perplexity/sonar-small-online", # Optional
            "messages": [
            {"role": "user", "content": "Give me the top three links to current news articles in a numbered list related to this tweet: " + text \
                + "make sure to included the title  followed by a colon and then the link to the article"}
            ]
        })
    )
    data = json.loads(response.text)
    message_content = data['choices'][0]['message']['content']

    def get_title_link(input_text):
        class get_title(dspy.Signature):
            """Given a string with a title in it, give me just the title as a string"""
            text = dspy.InputField()
            title = dspy.OutputField(desc="title of article")
        title = dspy.Predict(get_title)
        title_out = title(text=input_text).toDict()['title']

        class get_link(dspy.Signature):
            """Given a string with a link in it, give me just the link as a string"""
            text = dspy.InputField()
            link = dspy.OutputField(desc="link to article")
        link = dspy.Predict(get_link)
        link_out = link(text=input_text).toDict()['link']

        return {"title": title_out, "link": link_out}

    class format_output(dspy.Signature):
        """Given a string of articles, give me the title and link of each article"""
        text = dspy.InputField()
        formatted = dspy.OutputField(desc="title of article and link to article")
    formatted = dspy.Predict(format_output)
    answer = formatted(text=message_content)
    answer = answer.toDict()['formatted']
    lines = answer.split('\n')

    articles = []

    for line in lines:
        articles.append(get_title_link(line))

    class classify_party(dspy.Signature):
        """Given tweet text, classify the political party of the author"""
        text = dspy.InputField()
        party = dspy.OutputField(desc="Right, Left, or Center")
    party_output = dspy.Predict(classify_party)
    party = party_output(text=text).toDict()['party']
    
    
    return {"party": party, "articles": articles}

In [203]:
print(text_to_politics("January 6th was a dark day for America. We must never forget the insurrection."))

{'party': 'Left', 'articles': [{'title': 'January 6 insurrection news: Trials, investigations and more', 'link': 'https://www.nbcnews.com/january-6-insurrection-us-capitol'}, {'title': 'Republican lawmaker defends Trump endorsement despite Trump downplaying Jan. 6', 'link': 'https://abcnews.go.com/US/january6/'}, {'title': 'U.S. Capitol Riot', 'link': 'https://www.washingtonpost.com/january-6-capitol-riot/'}, {'title': 'Political turmoil exposed as representatives reveal their uncertainty on January 6', 'link': 'https://www.usatoday.com/story/news/politics/2021/01/09/january-6-capitol-violence-exposes-political-turmoil-members-congress-uncertain'}]}


In [103]:
turbo = dspy.OpenAI(model='gpt-4', api_key=OPENAI_API_KEY)
dspy.configure(lm=turbo)

In [196]:
text = """
1. **Donald Trump\'s Middle Eastern Improvements**: "https://www.cnn.com/2021/01/15/politics/trump-middle-east-policy" :)\n2. **After Capitol Riot, Republican Party Contends with Post-Trump Fragile Unity**: "https://www.washingtonpost.com/politics/republican-party-fragile-unity-after-capitol-riot/2020/12/23/db703c5c-a6f9-11eb-82d5-0035628ccf41_story.html" :)\n3. **January 6th Attack on the Capitol**: "https://www.npr.org/2021/01/16/957234700/january-6-attack-on-the-capitol-the-aftermath" :)
"""
class format_output(dspy.Signature):
    """Given a string of articles, give me the title and link of each article"""
    text = dspy.InputField()
    formatted = dspy.OutputField(desc="title of article and link to article")
formatted = dspy.Predict(format_output)
answer = formatted(text=text)
answer = answer.toDict()['formatted']
lines = answer.split('\n')


# class classify_party(dspy.Signature):
#     """Given tweet text, classify the political party of the author"""
#     text = dspy.InputField()
#     party = dspy.OutputField(desc="Right, Left, or Center")
# party = dspy.Predict(classify_party)
# answer = party(text="January 6th was a dark day for America. We must never forget the insurrection.")
# answer.toDict()['party']



["1. Title: Donald Trump's Middle Eastern Improvements, Link: https://www.cnn.com/2021/01/15/politics/trump-middle-east-policy",
 '2. Title: After Capitol Riot, Republican Party Contends with Post-Trump Fragile Unity, Link: https://www.washingtonpost.com/politics/republican-party-fragile-unity-after-capitol-riot/2020/12/23/db703c5c-a6f9-11eb-82d5-0035628ccf41_story.html',
 '3. Title: January 6th Attack on the Capitol, Link: https://www.npr.org/2021/01/16/957234700/january-6-']

In [192]:
def get_title_link(input_text):
    class get_title(dspy.Signature):
        """Given a string with a title in it, give me just the title as a string"""
        text = dspy.InputField()
        title = dspy.OutputField(desc="title of article")
    title = dspy.Predict(get_title)
    title_out = title(text=input_text).toDict()['title']

    class get_link(dspy.Signature):
        """Given a string with a link in it, give me just the link as a string"""
        text = dspy.InputField()
        link = dspy.OutputField(desc="link to article")
    link = dspy.Predict(get_link)
    link_out = link(text=input_text).toDict()['link']

    return {"title": title_out, "link": link_out}

get_title_link(lines[0])

{'title': "Donald Trump's Middle Eastern Improvements",
 'link': 'https://www.cnn.com/2021/01/15/politics/trump-middle-east-policy'}